In [1]:
# ─────────────────────────────────────────────────────────────
# 🔑  Cell 1 – Assume the cross-account role via STS
# ─────────────────────────────────────────────────────────────
import boto3, awswrangler as wr
from botocore.config import Config

TARGET_ROLE_ARN = (
    "arn:aws:iam::325699878264:role/"
    "TimestreamCrossAccountAccessRole-325699878264-us-west-2"
)
REGION = "us-west-2"
SESSION_NAME = "studio-timestream-xacct"

# Slightly higher read timeout to cope with long queries
cfg = Config(connect_timeout=5, read_timeout=60)

sts = boto3.client("sts", config=cfg)
creds = sts.assume_role(RoleArn=TARGET_ROLE_ARN,
                        RoleSessionName=SESSION_NAME)["Credentials"]

xacct = boto3.Session(
    aws_access_key_id     = creds["AccessKeyId"],
    aws_secret_access_key = creds["SecretAccessKey"],
    aws_session_token     = creds["SessionToken"],
    region_name           = REGION,
)

# Verify we’re in the right account
identity = xacct.client("sts").get_caller_identity()
print("Assumed role in account:", identity["Account"])

# ─────────────────────────────────────────────────────────────
# 🗂️  Cell 2 – Discover databases & tables (wr.timestream)
# ─────────────────────────────────────────────────────────────
dbs = wr.timestream.list_databases(boto3_session=xacct)
print("Databases:", dbs)

DB_NAME    = "prod-gem-plat-event-database"        # adjust if spelling differs
TABLE_NAME = "prod-gem-plat-event-table-partitioned"

tbls = wr.timestream.list_tables(database=DB_NAME, boto3_session=xacct)
print(f"Tables in {DB_NAME}:", tbls)

if TABLE_NAME not in tbls:
    raise ValueError(f"{TABLE_NAME} not found — double-check spelling/region")

# ─────────────────────────────────────────────────────────────
# 🔍  Cell 3 – Query the last 24 h of data into a DataFrame
# ─────────────────────────────────────────────────────────────
QUERY = f"""
SELECT *
FROM "{DB_NAME}"."{TABLE_NAME}"
WHERE time > ago(1d)       
LIMIT 10000               
"""

df = wr.timestream.query(sql=QUERY, boto3_session=xacct)
print(f"Returned {len(df):,} rows")
df.head()


Assumed role in account: 325699878264
Databases: ['prod-gem-plat-event-database']
Tables in prod-gem-plat-event-database: ['custom-ui-threshold-test', 'prod-gem-plat-alert-thresholds-table', 'prod-gem-plat-business-event-table', 'prod-gem-plat-cold-inflation-pressure-table', 'prod-gem-plat-custom-alert-parameters-table', 'prod-gem-plat-event-table-partitioned', 'prod-gem-plat-tire-event-table']
Returned 10,000 rows


/home/sagemaker-user/.conda/envs/SaroshEnv/lib/python3.10/site-packages/awswrangler/timestream/_read.py:169: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(results, ignore_index=True)


,tire_side,sample_end_timestamp,device_id,tire_position,fwv,hwv,halo_enabled,transmission_timestamp,axle_number,sample_start_timestamp,...,vehicle_battery_voltage,gps_altitude,gps_fix_sats,rssi,recvd,sample_distance_meters,sample_count,total_distance_meters,internal_battery_temp_c,occurred
0,<NA>,1747279465000,10090812,<NA>,24.12.12.08,device_pn=&device_rev=&pcba_pn=31-00009801&pcb...,<NA>,1747279920000,<NA>,1747278114000,...,NaN,NaN,NaN,NaN,NaN,6.0,4.0,154018.0,None,NaN
1,<NA>,<NA>,100D92C4,<NA>,24.12.12.08,device_pn=&device_rev=&pcba_pn=31-00009801&pcb...,<NA>,1747279921000,<NA>,<NA>,...,12.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2,<NA>,1747279857000,100D8D9E,<NA>,24.12.12.08,device_pn=&device_rev=&pcba_pn=31-00009801&pcb...,<NA>,1747279924000,<NA>,1747278202000,...,NaN,NaN,NaN,NaN,NaN,534.0,116.0,302519.0,None,NaN
3,<NA>,<NA>,100D8D9E,<NA>,24.12.12.08,device_pn=&device_rev=&pcba_pn=31-00009801&pcb...,<NA>,1747279921000,<NA>,<NA>,...,12.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
4,<NA>,0,1009124A,<NA>,24.12.12.08,device_pn=&device_rev=&pcba_pn=31-00009801&pcb...,<NA>,1747279921000,<NA>,0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,22120722.0,None,NaN
